# Problem Description.

This project is based in the section about Meta programming of the paper [1].
Before starting, we recommend you to read carefully that section, 
but you can skip the subsection on Guess-and-check.

We start with some preliminaries, and afterwards we describe the three parts of this project.

[1] Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2020). How to build your own ASP-based system?! CoRR, abs/2008.06692.

## Preliminaries.

Let us consider the Latin Square problem:
given a quadratic board of size `s`, 
the goal is to fill each cell of the
board with some number from `1` to `s` such that 
no number occurs twice in the same row or column.

This problem can be represented by the following logic program:

In [1]:
%%file latin.lp
number(1..s).

1 { latin(X,Y,N) : number(N) } 1 :- number(X), number(Y).
1 { latin(X,Y,N) : number(Y) } 1 :- number(X), number(N).
1 { latin(X,Y,N) : number(X) } 1 :- number(Y), number(N).

#show latin/3.

Writing latin.lp


The paper [1] presents the following meta encoding `meta.lp`.

In [3]:
%%file meta.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Overwriting meta.lp


The file can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by `clingo`.
For example, the answer sets generated by this command:

In [5]:
!clingo latin.lp -c s=4 0 --quiet

clingo version 5.4.0
Reading from latin.lp
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


are the same as those generated by these ones:

In [7]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
!clingo reify.lp meta.lp 0 --quiet

clingo version 5.4.0
Reading from reify.lp ...
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.030s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.030s


You can check this equivalence using the following Python function.
It takes as input a string and some files that, together, specify a logic program.
The function returns a sorted version of the answer sets of the program.

In [8]:
import clingo

def run(program="", files=[]):
    ctl = clingo.Control(["0"])
    ctl.add("base", [], program)
    for f in files:
        ctl.load(f)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        return sorted(sorted(list(m.symbols(shown=True))) for m in handle)

Note that the answer sets returned consist only of atoms and terms that are shown.

We can check that the previous programs have the same answer sets as follows:

In [9]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta.lp']) == run("#const s=4.", ['latin.lp'])

True

The comparison returns `True` because the answer sets of both program are the same.

Click the link at the following cell to download a zip file with all the relevant files for this project.·

In [ ]:
from IPython.display import FileLink
FileLink("meta-encoding.zip")

## Part 1. Translating choice rules and weight rules to normal rules.

The task of this part is to write a meta encoding `meta-normal.lp` 
whose ground instantiation (generated by `clingo`) has no choice rules or weight rules.

The new meta encoding `meta-normal.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by `clingo`.

In [2]:
%%file meta-normal.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
    
%
% Replace this rule:
%
% body(sum(B,G))  :- rule(_,sum(B,G)),
%     #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
%            W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).

%
% Replace this rule:
%
% { hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Writing meta-normal.lp


Once it is ready, you can run the same test as before:

In [ ]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta-normal.lp']) == run("#const s=4.", ['latin.lp'])

You can check if the ground instantiation of the meta encoding has no choice rules or weight rules with this call:

In [ ]:
!clingo latin.lp         -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-normal.lp --output=reify

If the meta encoding is correct, 
then the output should contain no facts of predicate `weighted_literal_tuple/3`, 
and no facts of the form `rule(choice(t1),t2)` for any terms `t1` and `t2`.

In the directory ``translation-instances`` you can find 10 instances. They consist of the `latin.lp` encoding together with one additional aggregate, whose bounds are different along the instances. Your meta encoding should work with all those instances. More specifically, for each instance `ins.lp`, the following cell should return `True`.

In [ ]:
!clingo ins.lp --output=reify > reify.lp
run("", ['reify.lp', 'meta-normal.lp']) == run("", ['ins.lp'])

### Translating choice rules

The meta-encoding `meta.lp` contains the rule
```
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).
```
that generates the atoms of a choice rule whenever its body holds.
The first step of this part is to replace that rule by a set of normal rules.

In our course Answer Set Solving in Practice (https://teaching.potassco.org), 
in the Language part, we explain how to do this for propositional logic programs.
You can apply the same approach here.

### Translating weight rules

The meta-encoding `meta.lp` contains the rule
```
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.
```
that determines when the body of a weight rule holds, using a `#sum` aggregate in the body.
The second step of this part is to replace that rule by a set of normal rules.

In our course Answer Set Solving in Practice (https://teaching.potassco.org),
in the Language part, 
we explain how to do this for cardinality rules occurring in propositional logic programs.
You can apply the same approach here, except that in this case, 
whenever a literal holds, 
instead of adding `1` to the accumulated values, 
we have to add the corresponding weight `w` of the literal.

The approach requires to order the literals occurring in the weight rules.
That can be done with this rule:
```
weighted_literal_tuple(B,L,W,P) :- weighted_literal_tuple(B,L,W),
                                   P = #count{ LL,WW : weighted_literal_tuple(B,LL,WW), (LL,WW)<=(L,W) }. 
```
The argument `P` of `weighted_literal_tuple` gives the order of the literal `L` with weight `W` in the weight rule `B`. Observe that the rule uses an aggregate, 
but since it only depends on the facts about `weighted_literal_tuple/3`,
`clingo` can simplify the rule and ground it into facts.  
The rule is not normal, 
but given that its ground instantiation has no choice rules or weight rules, 
you can (and should) use it in your meta encoding.

**Note**: The weights generated by `clingo` in the reified output are always positive integers.
In other words, the third argument of the atoms of the predicate `weighted_literal_tuple/3` is always a positive integer.

## Part 2. Translating tight normal programs to choice rules and integrity constraints.

The task of this part is to write a meta encoding `meta-constraints.lp` 
whose ground instantiation (generated by `clingo`) has only choice rules and integrity constraints.

We restrict the input programs to normal programs with integrity constraints.
These programs do not contain disjunctions, choice rules or weight rules. 
Furthermore, the input programs have to be tight, i.e., 
the positive atom dependency graph of the ground instantiation of the programs must be acyclic. 

Note that these are the programs that we obtain from Part 1
if the original program is tight and non-disjunctive, 
as in the case of `latin.lp`.
To see this, please inspect the output of the next cell:

In [ ]:
!clingo latin.lp         -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-normal.lp --output=reify

The new meta encoding `meta-constraints.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a tight normal logic program `P` with integrity constraints,
given a reified version of the program `P` generated by `clingo`.
You can run a similar test as before:

In [ ]:
!clingo latin.lp          -c s=4 --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
run("", ['reify2.lp', 'meta-constraints.lp']) == run("#const s=4.", ['latin.lp'])

In the course Answer Set Solving in Practice (https://teaching.potassco.org), 
in the Solving part, 
we show how to represent a tight logic program by a set of nogoods
such that the answer sets of the logic program correspond 
to the solutions to the nogoods.
For tight logic programs, those nogoods are called the completion nogoods.

The answer sets of the meta encoding `meta-constraints.lp` should correspond to 
the solutions to the completion nogoods.
The choice rules are used to generate all possible assignments, while
the constraints represent the completion nogoods.
For example, the nogood `{Ta, Fb, Fc}` can be represented by the constraint `:- a, not b, not c.`.
Then, given the previous result from the course, 
the answer sets of the meta encoding will correspond to the answer sets of the original logic program, 
and this part will be completed.

We already provide you with the choice rules and the show statements that you need:

In [3]:
%%file meta-constraints.lp

{conjunction(B)} :- literal_tuple(B).
{     hold(|L|)} :- literal_tuple(B,L).
{     hold( A )} :- atom_tuple(H,A).

% Add the constraints here...
    
#show.
#show T : output(T,B), conjunction(B).

Writing meta-constraints.lp


Your task is to add the constraints representing the nogoods.
You only need two constraints for the atom-oriented nogoods, and
three constraints for the body-oriented nogoods.

Your meta encoding should work with the 10 instances of the directory ``translation-instances``. More specifically, for each instance `ins.lp`, the following cell should return `True`.

In [ ]:
!clingo ins.lp                   --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
run("", ['reify2.lp', 'meta-constraints.lp']) == run("", ['ins.lp'])

## Part 3. Computing weighted diverse optimal models 

The paper [1] presents the following meta encoding for computing several diverse answer sets:

In [1]:
%%file many.lp
model(1..m).

conjunction(B,M) :- model(M), literal_tuple(B),
        hold(L,M) : literal_tuple(B, L), L > 0;
    not hold(L,M) : literal_tuple(B,-L), L > 0.

body(normal(B),M) :- rule(_,normal(B)), conjunction(B,M).
body(sum(B,G),M)  :- model(M), rule(_,sum(B,G)),
    #sum { W,L :     hold(L,M), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L,M), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A,M) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B,M).
{ hold(A,M) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B,M).

show(T,M) :- output(T,B), conjunction(B,M).
#show.
#show (T,M) : show(T,M).

#const k=1.

:- model(M), model(N), M<N, option = 1,
    #sum{ 1,T: show(T,M), not show(T,N) ;
          1,T: not show(T,M), show(T,N) } < k.

#maximize{ 1,T,M,N: show(T,M), not show(T,N), model(N), option = 2 }.

Writing many.lp


The task of this part is to write a meta encoding `weighted-many.lp`, 
extending `many.lp`,
for computing weighted diverse answer sets.

In this case the input logic program is accompanied by 
a set of facts about predicate `weight/2`, assigning weights to atoms:
```
weight(A,W). % The weight of atom A is W
```
Then the distance between two answer sets is the sum of the weights of the atoms 
that are interpreted differently in the two answer sets.
The notions of `k`-diverse and most-diverse answer sets are defined as in [1], 
using the new distance measure instead of the Hamming distance.
As a result, the notions of [1] correspond to the special case of this setting
where all atoms are given a weight of `1`.


In the directory `diverse-instances` you can find 10 instances. 
They consist of the `cells.lp` encoding of [1] (using a `3x3` grid and marking `7` cells) 
together with different sets of facts about predicate `weight/2`.
Your meta encoding should work with all those instances. 
More specifically, for each instance `ins.lp`, the following command should return the sets of `3` weighted most-diverse answer sets (they appear commented at the end of every instance file):

In [ ]:
!clingo ins.lp --output=reify > reify.lp
!clingo reify.lp weighted-many.lp -c m=3 -c option=2 --opt-mode=optN 0 --quiet=1